### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [106]:
from matplotlib import pyplot as plt
import os
import numpy as np
import cv2 # OpenCV

In [107]:
folderpath = './images'

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [108]:
grid_size = (8,11)
def calculate_homography(image)->tuple[np.ndarray, np.ndarray, np.ndarray]:
    return_value, corners = cv2.findChessboardCorners(image, patternSize=grid_size)
    assert(return_value)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    corners = corners.reshape((88,2)).copy()
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), criteria)
    A = np.empty((0,9), dtype=float)
    O = np.array([0,0,0]).reshape(1,3)
    square_size = 11/1000 # meters
    real_coords = []
    pix_coords = []
    for index, corner in enumerate(corners):
        Xpix = corner[0] # in pixel
        Ypix = corner[1]
        pix_coords.append(np.array([Xpix, Ypix]))
        grid_size_cv2 = tuple(reversed(grid_size)) # OpenCV and Python store matrices differently
        u_index, v_index = np.unravel_index(index, grid_size_cv2) # convert index from linear to 2D (0-based indexing)
        Xmm = u_index * square_size
        Ymm = v_index * square_size
        real_coords.append(np.array([Xmm, Ymm, 0, 1]))
        m = np.array([Xmm, Ymm, 1]).reshape(1,3)
        A = np.vstack((A, np.hstack((m,O,-Xpix*m))))
        A = np.vstack((A, np.hstack((O,m,-Ypix*m))))

    _, _, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:,-1]
    H = h.reshape(3,3)
    return H, np.array(real_coords), np.array(pix_coords)

In [109]:
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()

In [110]:
HH = []
for path in images_path:
    image = cv2.imread(path)
    H, _, _ = calculate_homography(image)
    HH.append(H)

2. per ogni omografia dalla relativa matrice costruire la matrice V + fare funzione che costruisce i vettori v, i, j

In [111]:
def vij_function (H, i, j):
  v=np.zeros(6)
  v[0]=H[0][i]*H[0][j]
  v[1]=H[0][i]*H[1][j]+H[1][i]*H[0][j]
  v[2]=H[1][i]*H[1][j]
  v[3]=H[2][i]*H[0][j]+H[0][i]*H[2][j]
  v[4]=H[2][i]*H[1][j]+H[1][i]*H[2][j]
  v[5]=H[2][i]*H[2][j]
  return v

In [112]:
VV=np.zeros((0,6))

for H in HH:
  v11=vij_function(H, 0, 0)
  v12=vij_function(H, 0, 1)
  v22=vij_function(H, 1, 1)
  V=np.array([v12.T, (v11-v22).T])
  VV=np.vstack((VV, V))

3. data V trovare B applicando la singular value decomposition

In [113]:
U, Sigma, S_T = np.linalg.svd(VV)
b = S_T.transpose()[:, -1]
B = np.empty((3, 3))
B[0, 0] = b[0]
B[0, 1] = B[1, 0] = b[1]
B[1, 1] = b[2]
B[0, 2] = B[2, 0] = b[3]
B[2, 1] = B[1, 2] = b[4]
B[2, 2] = b[5]
print(B)
eigvals, _ = np.linalg.eigh(B)
print(eigvals)

[[-2.66202724e-07  1.59809311e-10  1.68223842e-04]
 [ 1.59809311e-10 -2.65953699e-07  1.32339922e-04]
 [ 1.68223842e-04  1.32339922e-04 -9.99999977e-01]]
[-1.00000002e+00 -2.66204763e-07 -2.20138534e-07]


4. da B ottenere K e di conseguenza trovare R

In [114]:
def calculate_projection_matrix(H:np.ndarray, K:np.ndarray)->np.ndarray:
    K_inv = np.linalg.inv(K)
    h1 = H[0, :]
    h2 = H[1, :]
    h3 = H[2, :]
    lambdas = 1/np.linalg.norm(K_inv@h1)
    r1 = lambdas * K_inv@h1
    r2 = lambdas * K_inv@h2
    t = lambdas * K_inv@h3
    t = t.reshape((-1, 1))
    r3 = np.cross(r1, r2)
    R = np.hstack((r1.reshape((-1, 1)), r2.reshape((-1, 1)), r3.reshape((-1, 1))))
    U, _ , V_t = np.linalg.svd(R)
    R = U@V_t
    P = K@np.hstack((R, t))
    return P

In [115]:
B11 = b[0]
B12 = b[1]
B22 = b[2]
B13 = b[3]
B23 = b[4]
B33 = b[5]

v0 = (B12 * B13 - B11 * B23)/(B11 * B22 - B12 * B12)
l = B33 - (B13**2 + v0*(B12 * B13 - B11 * B13))/B11
alpha = np.sqrt(l/B11)
beta = np.sqrt(l * B11/(B11 * B22 - B12**2))
gamma = - B12 * (alpha**2) * beta / l
u0 = gamma * v0 / beta - B13 * (alpha**2) / l

K = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]])

K_inv = np.linalg.inv(K)
print("K:",K)
projections = []

for i, H in enumerate(HH):
    P = calculate_projection_matrix(H, K)
    projections.append(P)

print(projections[0])

K: [[1.74422010e+03 1.04759684e+00 6.32237799e+02]
 [0.00000000e+00 1.74503682e+03 4.97985025e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 9.99407891e+02 -1.43052020e+03 -6.29940735e+02 -8.27585326e-05]
 [-2.61289734e+02  3.69917133e+02 -1.75727960e+03  4.11018439e-04]
 [ 8.80149846e-01  3.25211118e-01 -3.45794704e-01  9.95315863e-04]]


5. scegliamo un immagine, abbiamo bisogno di matrice di proiezione per l'immagine e i punti reali e proiettati dei corners della scacchiera che possiamo ricavare. Calcoliamo il reprojection error (pagina 45 lecture 3) (somma per ogni punto)

In [116]:
#reprojection error
def reprojection_error (P, R, I):
  epsilon_tot=0
  for i in range(R.shape[0]):
    u = np.dot(P[0], R[i])/np.dot(P[2], R[i])
    v = np.dot(P[1], R[i])/np.dot(P[2], R[i])

    print(I[i], u, v)
    epsilon = ((np.dot(P[0], R[i]) / np.dot(P[2], R[i]))-I[i][0])**2 + ((np.dot(P[1], R[i]) / np.dot(P[2], R[i]))-I[i][1])**2
    print(epsilon)
    epsilon_tot += epsilon
  return epsilon_tot

In [117]:
image = cv2.imread(images_path[0])
H, R, I = calculate_homography(image)
R = R
I = I
P = calculate_projection_matrix(H, K)
error = reprojection_error(P, R, I)
print(error)

[912.19885 637.3455 ] -0.08314800926349475 0.41295276626088057
1237941.5437522978
[879.38477 624.95764] -3441.296773389128 889.9675280907576
18738519.2015145
[846.91705 612.63544] -3861.555807782949 998.6045926743931
22318688.871949267
[814.66864 600.58795] -4025.4209194455084 1040.963764665466
23620397.80145697
[783.023  588.6812] -4112.6818566568745 1063.520738568795
24193398.718576465
[751.4288 576.7618] -4166.878320038279 1077.5305377895606
24440514.000318334
[720.1944  564.96674] -4203.809895239565 1087.0773591353636
24518417.772682413
[688.91705 553.21295] -4230.59296489099 1094.0007892804567
24494030.303638563
[924.11096 604.799  ] 1029.6376260164475 -269.15666395031747
774934.3990172815
[891.35504 592.6011 ] -332.6941797559738 83.85634782667475
1757117.6926986666
[858.73315 580.4869 ] -1148.4123840125274 295.2285985704334
4110005.4976197705
[826.4818 568.5113] -1691.525953192283 435.96241685223714
6357932.307340083
[794.62427 556.6728 ] -2079.12262652576 536.3980584693797
82588

6. superimposition di un oggetto per tutte le immagini